In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

In [2]:
%matplotlib inline

## Generating Data

In [3]:
seed = 6343
np.random.seed(seed)

In [4]:
n = 15
m = 15

In [5]:
farm_clusters = (np.random.random((n, 2)) * 10).round(decimals=1)
ranges = np.random.random((n, 2)) * 0.3

farm_pixels = []
for i in range(n):
    xs = farm_clusters[i][0] - ranges[i][0]
    xe = farm_clusters[i][0] + ranges[i][0]

    ys = farm_clusters[i][1] - ranges[i][1]
    ye = farm_clusters[i][1] + ranges[i][1]

    for x in np.arange(xs, xe + 0.1, 0.1):
        for y in np.arange(ys, ye + 0.1, 0.1):
            farm_pixels.append([x, y])

farm_pixels = np.array(farm_pixels).round(decimals=1)
farm_pixels = farm_pixels[~((farm_pixels[:, 0] < 0) | (farm_pixels[:, 1] < 0))]

wells = (np.random.random((m, 2)) * 10).round(decimals=1)

In [6]:
seasonal_factor = [-1, -2, 3, 0]

In [7]:
base_water_level = -np.random.randint(0, n)
distance_weight = np.random.random()
farm_factor = np.random.random()

In [8]:
def get_observations(
    wells, farms, farm_factor, distance_weight, base_water_level, seasonal_factor
):
    def pairwise_distances(x, y=None):
        x_norm = (x ** 2).sum(1)[:, None]
        y_norm = (y ** 2).sum(1)[None, :]

        dist = x_norm + y_norm - 2.0 * np.matmul(x, y.T)

        return dist

    def get_observations_at_timestep(wells, farms, base):
        obs = (
            base
            - np.sum(np.exp(-pairwise_distances(wells, farms) / distance_weight), 1)
            * farm_factor
        )
        obs += np.random.randn(*obs.shape) * 0.5

        return obs

    base_water_levels = list()
    for i in range(40):
        base_water_level = min(
            base_water_level
            + seasonal_factor[i % 4]
            + np.random.randn() / 5,
            0,
        )

        base_water_levels.append(base_water_level)
        
    parameters = {"mu_{}".format(i): base_water_levels[i] for i in range(40)}
    
    parameters["delta"] = farm_factor
    parameters["theta_f"] = distance_weight
    
    parameters["sf"] = seasonal_factor

    return parameters, [
        get_observations_at_timestep(wells, farms, base_water_levels[i])
        for i in range(40)
    ]

In [9]:
parameters, observations = get_observations(
    wells, farm_pixels, farm_factor, distance_weight, base_water_level, seasonal_factor
)

with open("data/sample_data2.csv", "w") as f:
    f.write("timestep,type,latitude,longitude,observation\n")

    f.writelines(["0,farm,{},{},1\n".format(x, y) for x, y in farm_pixels])
    
    for t, observations_ in enumerate(observations):
        f.writelines(
            ["{},well,{},{},{}\n".format(t, x, y, o) for (x, y), o in zip(wells, observations_)]
        )

In [10]:
plt.clf()
fig = plt.figure(figsize=(10, 10), dpi=100)

plt.ion()

plt.scatter(farm_pixels[:, 0], farm_pixels[:, 1], marker="s", s=5, color="lightgreen")

scat = plt.scatter(wells[:, 0], wells[:, 1], marker="s", s=5, c=[(0, 0, 0, 1)] * len(wells))
label = plt.text(0, 0, '', fontsize=7)

colors = []
for observations_ in observations:
    min_v = min(observations_)
    max_v = max(observations_)
#     colors.append([max((x - min_v) / (max_v - min_v), 0.1) for x in observations_])
    colors.append([1 - abs(x) / 25 for x in observations_])
    
colors = np.array(colors)

def update_plot(i, scat):
    scat.set_array(colors[i])
    label.set_text(["Sp", "Su", "Fa", "Wi"][i % 4])
    return scat,

ani = animation.FuncAnimation(fig, update_plot, frames=range(len(observations)), fargs=(scat,), interval=1000)

plt.gray()
plt.close();

<Figure size 432x288 with 0 Axes>

In [11]:
HTML(ani.to_html5_video())

In [12]:
with open("data/gp-samples-t.json", "r") as f:
    samples = {k: np.array(v) for k, v in json.load(f).items()}

In [13]:
keys = set(samples.keys()).intersection(parameters.keys())

In [15]:
for k in sorted(list(keys)):
    print(k, " =\t", samples[k].mean(0), parameters[k])

delta  =	 0.6928055089712143 0.6923988343700731
mu_0  =	 -5.287082362174988 -5.170894281715216
mu_1  =	 -7.090107460021972 -7.1748215840154295
mu_10  =	 -4.824023308753968 -4.897075790337606
mu_11  =	 -5.167386012077332 -5.100981229013156
mu_12  =	 -6.239235377311706 -6.0181268937917896
mu_13  =	 -8.193080224990844 -8.037181330315658
mu_14  =	 -5.113154845237732 -4.970195471406862
mu_15  =	 -4.644397449493408 -4.773822098416652
mu_16  =	 -5.318944754600525 -5.358511908981449
mu_17  =	 -7.3959481191635135 -7.312134766312091
mu_18  =	 -4.568122057914734 -4.532401627333797
mu_19  =	 -4.293840537071228 -4.374659216003001
mu_2  =	 -4.335941343307495 -4.320061730571852
mu_20  =	 -5.6745987176895145 -5.778596019880883
mu_21  =	 -7.779528813362122 -7.815357556769764
mu_22  =	 -4.493146376609802 -4.4518905104042155
mu_23  =	 -4.9227232122421265 -4.877874801617113
mu_24  =	 -5.647332119941711 -5.723535343499913
mu_25  =	 -7.761023354530335 -7.751184344371361
mu_26  =	 -4.702457165718078 -4.64735